# Notebook to select host galaxies and submit candidates to TNS

In [1]:
import sys, os
import pandas as pd
import numpy as np
import ipyaladin as ipyal
from ipywidgets import Layout, Box, widgets

In [2]:
from astroquery.ned import Ned
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
import astropy.units as u
from astropy.table import Table, Column

In [3]:
sys.path.append("./lib")
from alerce_tns import *

In [4]:
# continuously load modules (for easier debugging)
%load_ext autoreload
%autoreload 2

# Start API

In [5]:
alerce = alerce_tns()

# Load recent candidates

# CHANGE DATE

In [6]:
refstring = "20191113"
candidates_dir = "candidates"
candidates = open("%s/%s.txt" % (candidates_dir, refstring)).read().splitlines()
print(candidates)

['ZTF19acryaon', 'ZTF19acryurj', 'ZTF19acrywgb', 'ZTF19acryynr', 'ZTF19acrzbwl', 'ZTF19acrzcgx', 'ZTF19acsamnm', 'ZTF19acmxymo', 'ZTF19acmyacs', 'ZTF19ackjvtl', 'ZTF19ackjuvd', 'ZTF19acsbxem', 'ZTF19acsbxgw', 'ZTF19ackjtpd', 'ZTF18aaknyrs']


# Select host galaxies using aladin widget

In [7]:
alerce.select_hosts(candidates, catsHTM=False, vizier=False)

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

/home/fforster/anaconda3/lib/python3.6/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '2:03:33.6737 -9:06:42.1639': No astronomical object found :
  (error.line, error.msg))


Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

...

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

/home/fforster/anaconda3/lib/python3.6/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '12:31:10.0941 +19:34:56.2483': No astronomical object found :
  (error.line, error.msg))


Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

In [8]:
alerce.candidate_host_names

{'ZTF19acryaon': 'WISEA J005749.28-173826.7',
 'ZTF19acryurj': 'UGC 00548',
 'ZTF19acrywgb': 'SDSS J020333.76-090643.2',
 'ZTF19acryynr': 'UGC 01046',
 'ZTF19acrzbwl': 'WISEA J020701.29+112721.1',
 'ZTF19acrzcgx': 'SDSS J015708.94+151437.3',
 'ZTF19acsamnm': 'WISEA J063100.18+375425.1',
 'ZTF19acmxymo': 'WISEA J120719.68+065056.4',
 'ZTF19acmyacs': 'NGC 4793',
 'ZTF19ackjvtl': 'KUG 1313+309',
 'ZTF19ackjuvd': 'WISEA J125819.97+311238.4',
 'ZTF19acsbxem': 'NGC 4954',
 'ZTF19acsbxgw': 'WISEA J125125.75+700734.8',
 'ZTF19ackjtpd': 'SDSS J123110.20+193452.8',
 'ZTF18aaknyrs': 'WISEA J121431.17+181504.5'}

# Manual modifications

In [9]:
#oid = "ZTF19acmbjll"
#alerce.candidate_host_names[oid] = "WISEA J052037.89-090209.7"
#alerce.candidate_host_redshifts[oid] = "nan"

# Clean nans

In [10]:
for k in list(alerce.candidate_host_redshifts.keys()):
    if alerce.candidate_host_redshifts[k] == "nan":
        alerce.candidate_host_redshifts.pop(k)

# Save hosts

In [11]:
df_host_names = pd.DataFrame.from_dict(alerce.candidate_host_names, orient='index', columns=["host_id"])
df_host_names.index.name='oid'
df_host_redshifts = pd.DataFrame.from_dict(alerce.candidate_host_redshifts, orient='index', columns=["z"])
df_host_redshifts.index.name='oid'
df_hosts = df_host_names.join(df_host_redshifts)
df_hosts.to_csv("candidates/%s_hosts.csv" % refstring)
df_hosts.head(100)

,host_id,z
oid,,
ZTF19acryaon,WISEA J005749.28-173826.7,0.055108
ZTF19acryurj,UGC 00548,0.022035
ZTF19acrywgb,SDSS J020333.76-090643.2,0.157
ZTF19acryynr,UGC 01046,0.017425
ZTF19acrzbwl,WISEA J020701.29+112721.1,NaN
ZTF19acrzcgx,SDSS J015708.94+151437.3,0.179
ZTF19acsamnm,WISEA J063100.18+375425.1,0.022816
ZTF19acmxymo,WISEA J120719.68+065056.4,0.075337
ZTF19acmyacs,NGC 4793,0.008286


# Display host galaxy properties

In [12]:
for oid in candidates:
    if oid in alerce.candidate_host_redshifts.keys():
        print(oid, alerce.candidate_host_names[oid], alerce.candidate_host_redshifts[oid])
    elif oid in alerce.candidate_host_names.keys():
        print(oid, alerce.candidate_host_names[oid])
    else:
        alerce.candidate_host_names[oid] = ""
        alerce.candidate_host_redshifts[oid] = ""
        print(oid)

ZTF19acryaon WISEA J005749.28-173826.7 0.055108
ZTF19acryurj UGC 00548 0.022035
ZTF19acrywgb SDSS J020333.76-090643.2 0.157
ZTF19acryynr UGC 01046 0.017425
ZTF19acrzbwl WISEA J020701.29+112721.1
ZTF19acrzcgx SDSS J015708.94+151437.3 0.179
ZTF19acsamnm WISEA J063100.18+375425.1 0.022816
ZTF19acmxymo WISEA J120719.68+065056.4 0.075337
ZTF19acmyacs NGC 4793 0.008286
ZTF19ackjvtl KUG 1313+309 0.018883
ZTF19ackjuvd WISEA J125819.97+311238.4 0.101125
ZTF19acsbxem NGC 4954 0.030961
ZTF19acsbxgw WISEA J125125.75+700734.8
ZTF19ackjtpd SDSS J123110.20+193452.8 0.052
ZTF18aaknyrs WISEA J121431.17+181504.5 0.029177


# Sandbox or real

In [13]:
sandbox = False # True: test in sandbox, False: send to real website

# Get API key and url

In [14]:
if sandbox:
    print("Getting API sandbox key...")
    api_key = open("API_sandbox.key", "r").read()
else:
    print("Getting API key...")
    api_key = open("API.key", "r").read()
    
if sandbox:
    url_tns_api="https://sandbox-tns.weizmann.ac.il/api"
else:
    url_tns_api="https://wis-tns.weizmann.ac.il/api"
    
if not sandbox:
    print("WARNING: submitting to real (not sandbox) TNS website")

Getting API key...


# Prepare TNS report

# CHECK AUTHOR LIST!!!
(key is used to query only at this step)

In [21]:
reporter = "F. Forster, F.E. Bauer, G. Pignata, J. Arredondo, G. Cabrera-Vives, R. Carrasco-Davis, P.A. Estevez, P. Huijse, E. Reyes, I. Reyes, P. Sanchez-Saez, C. Valenzuela, E. Castillo, D. Ruz-Mieres, D. Rodriguez-Mancini, F.E. Bauer, M. Catelan, S. Eyheramendy, M.J. Graham on behalf of the ALeRCE broker"
report = {"at_report": {}}
counter = 0
for oid in candidates:
    #if oid not in test:#["ZTF19acgjpfd", "ZTF18acszatf", "ZTF18acrkmnm"]:
    #    continue
    #if oid != "ZTF19acmxymo": #ZTF19acnqqhl":#in ["ZTF19acmdwjy", "ZTF19acmdwjz"]:
    #    continue
    print("\n", counter, oid)
    sn_report = alerce.do_TNS_report(api_key, oid, reporter, verbose=False, test=False)
    if sn_report:
        report["at_report"]["%i" % counter] = sn_report
        counter += 1


 0 ZTF19acryaon
http://alerce.online/object/ZTF19acryaon


Early SN candidate (g-rise > 0.14+-0.01 mag/day, r-rise > 0.12+-0.02 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19acryaon 
Astronomical transient is known: [{'objname': '2019urn', 'prefix': 'AT'}] 


 0 ZTF19acryurj
http://alerce.online/object/ZTF19acryurj


Early SN candidate (r-rise > 0.13+-0.02 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19acryurj 
Astronomical transient is known: [{'objname': '2019ust', 'prefix': 'AT'}] 


 0 ZTF19acrywgb
http://alerce.online/object/ZTF19acrywgb


Early SN candidate (r-rise > 0.11+-0.02 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19acrywgb 

 1 ZTF19acryynr
http://alerce.online/object/ZTF19acryynr


Early SN candidate (r-rise > 0.10+-0.02 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19acryynr 

 2 ZTF19acrzbwl
http://alerce.online/object/ZTF19acrzbwl


...

SN candidate classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19ackjuvd 

 8 ZTF19acsbxem
http://alerce.online/object/ZTF19acsbxem


SN candidate classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19acsbxem 

 9 ZTF19acsbxgw
http://alerce.online/object/ZTF19acsbxgw


SN candidate classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19acsbxgw 
Astronomical transient is known: [{'objname': '2019quv', 'prefix': 'AT'}] 


 9 ZTF19ackjtpd
http://alerce.online/object/ZTF19ackjtpd


SN candidate classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19ackjtpd 

 10 ZTF18aaknyrs
http://alerce.online/object/ZTF18aaknyrs


Early SN candidate (r-rise > 0.39+-0.08 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF18aaknyrs 
Astronomical transient is known: [{'objname': '2018lqp', 'prefix': 'AT'}] 



# Display report

In [22]:
print(json.dumps(report, indent=4, sort_keys=True))

{
    "at_report": {
        "0": {
            "at_type": "1",
            "dec": {
                "error": "0.09",
                "units": "arcsec",
                "value": "-9.1117122"
            },
            "discovery_datetime": "2019-11-13 07:04:14.002",
            "groupid": "74",
            "host_name": "SDSS J020333.76-090643.2",
            "host_redshift": "0.157",
            "internal_name": "ZTF19acrywgb",
            "non_detection": {
                "comments": "Data provided by ZTF",
                "exptime": "30",
                "filter_value": "110",
                "flux_units": "1",
                "instrument_value": "196",
                "limiting_flux": "18.8815002441406",
                "obsdate": "2019-11-13 05:32:37.000",
                "observer": "Robot"
            },
            "photometry": {
                "photometry_group": {
                    "0": {
                        "comments": "Data provided by ZTF",
                        

# Save report

In [20]:
json_report = '%s/%s.json' % (candidates_dir, refstring)
with open(json_report, 'w') as outfile:
    json.dump(report, outfile)

# Send to TNS

**WARNING: key is used to send candidates!**

In [21]:
json_report

'candidates/20191108.json'

In [22]:
r = alerce.send_json_report(api_key, url_tns_api, json_report)

# Show output

In [23]:
r.content

b'{"id_code":200,"id_message":"OK","data":{"report_id":26914,"recieved_data":{"at_report":{"0":{"ra":{"value":327.9856451,"error":0.09,"units":"arcsec"},"dec":{"value":-26.4084449,"error":0.09,"units":"arcsec"},"groupid":74,"reporter":"F. Forster, F.E. Bauer, G. Pignata, J. Arredondo, G. Cabrera-Vives, R. Carrasco-Davis, P.A. Estevez, P. Huijse, E. Reyes, I. Reyes, P. Sanchez-Saez, C. Valenzuela, E. Castillo, D. Ruz-Mieres, D. Rodriguez-Mancini, F.E. Bauer, M. Catelan, S. Eyheramendy, M.J. Graham on behalf of the ALeRCE broker","discovery_datetime":"2019-11-08 02:01:12.996","at_type":1,"internal_name":"ZTF19acqnabu","remarks":"SN candidate classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http:\\/\\/alerce.online\\/object\\/ZTF19acqnabu ","non_detection":{"obsdate":"2019-11-02 05:02:42.999","limiting_flux":19.216999053955,"flux_units":1,"filter_value":111,"instrument_value":196,"exptime":30,"observer":"Robot","comments":"Data provided by ZTF"},"photom

# Check the following [link](https://wis-tns.weizmann.ac.il/search?&discovered_period_value=1&discovered_period_units=months&unclassified_at=0&classified_sne=0&name=&name_like=0&isTNS_AT=all&public=all&ra=&decl=&radius=&coords_unit=arcsec&groupid%5B%5D=74&classifier_groupid%5B%5D=null&objtype%5B%5D=null&at_type%5B%5D=null&date_start%5Bdate%5D=&date_end%5Bdate%5D=&discovery_mag_min=&discovery_mag_max=&internal_name=&discoverer=&classifier=&spectra_count=&redshift_min=&redshift_max=&hostname=&ext_catid=&ra_range_min=&ra_range_max=&decl_range_min=&decl_range_max=&discovery_instrument%5B%5D=null&classification_instrument%5B%5D=null&associated_groups%5B%5D=null&at_rep_remarks=&class_rep_remarks=&num_page=50) and confirm that the latest SNe are from the reported day

https://wis-tns.weizmann.ac.il/search